<h3>Étape 1: Installation des bibliothèques requises

In [6]:
#conda install requests
#conda install html5lib
#conda install bs4

<h3>Étape 2: Analyse du contenu HTML depuis la page de commentaires de base

In [7]:
import requests
URL = "https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+"<product_id>"
r = requests.get(URL)
#print(r.content)

<h4><li>Importation de liste d'identifiants de produits</li></h4>

In [4]:
import pandas as pd
df = pd.read_csv('ids_reference.csv')
ids_list = list(df["product_id"])
review_number_list =list(df["reviews_number"])
df.head()


,product_id,reviews_number
0,WH804CP0T2ER8NAFAMZ,662
1,WH804FF1B9KBSNAFAMZ,168
2,BR884ST1G4KHCNAFAMZ,458
3,SA050CP0RTP6SNAFAMZ,26
4,WH804FF1H7WDKNAFAMZ,101


<h4><li>Analyse du contenu HTML de chaque produits</li></h4>

In [14]:
from bs4 import BeautifulSoup
users_data_list =[]
id_generator = 0
for i in ids_list:
    URL ="https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+str(i)
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    table = soup.find('div', attrs = {'class':'cola -phm -df -d-co'})
    if(table):
        for row in table.findAll('article',attrs = {'class':'-pvs -hr _bet'}):
            user ={}
            user["user_id"] = id_generator
            id_generator = id_generator+1
            user_name_full =row.find('span', attrs = {'class':''}).text
            user["user_name"] =user_name_full[4:(len(user_name_full))]
            user["product_id"] = i
            user_note_full = row.find('div', attrs = {'class' : 'stars _m _al -mvs'}).text
            user["user_note"] = user_note_full[0:1]
            user["user_comment"] = row.find('h3').text
            users_data_list.append(user)
    print(ids_list.index(i))

'"from bs4 import BeautifulSoup\nusers_data_list =[]\nid_generator = 0\nfor i in ids_list:\n    URL ="https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+str(i)\n    r = requests.get(URL)\n    soup = BeautifulSoup(r.content, \'html5lib\')\n    table = soup.find(\'div\', attrs = {\'class\':\'cola -phm -df -d-co\'})\n    if(table):\n        for row in table.findAll(\'article\',attrs = {\'class\':\'-pvs -hr _bet\'}):\n            user ={}\n            user["user_id"] = id_generator\n            id_generator = id_generator+1\n            user_name_full =row.find(\'span\', attrs = {\'class\':\'\'}).text\n            user["user_name"] =user_name_full[4:(len(user_name_full))]\n            user["product_id"] = i\n            user_note_full = row.find(\'div\', attrs = {\'class\' : \'stars _m _al -mvs\'}).text\n            user["user_note"] = user_note_full[0:1]\n            user["user_comment"] = row.find(\'h3\').text\n            users_data_list.append(user)\n    print(ids_list.index(

<h4><li>Enregistrement des données dans un fichier CSV

In [12]:
import csv
filename = 'users_ratings.csv'
with open(filename, 'w', newline='', encoding="utf-8") as f:
    w = csv.DictWriter(f,['user_id','user_name','product_id','user_note', 'user_comment'])
    w.writeheader()
    for user in users_data_list:
        w.writerow(user)

In [13]:
import pandas as pd
df = pd.read_csv("users_ratings.csv")
df.head(10)

,user_id,user_name,product_id,user_note,user_comment
0,0,Ayman,WH804CP0T2ER8NAFAMZ,5,J’aime ça
1,1,Fatima,WH804CP0T2ER8NAFAMZ,4,Produit chinois
2,2,Sedki,WH804CP0T2ER8NAFAMZ,4,J'aime
3,3,Salem,WH804CP0T2ER8NAFAMZ,5,J'aime
4,4,Mohamed Aymen,WH804CP0T2ER8NAFAMZ,4,bonne qualité
5,5,Iyed,WH804CP0T2ER8NAFAMZ,5,tres bienn
6,6,Alaeddine,WH804CP0T2ER8NAFAMZ,5,j'aime
7,7,Mohamed riadh,WH804CP0T2ER8NAFAMZ,5,Excellent
8,8,Abou Rouaa,WH804CP0T2ER8NAFAMZ,4,bon rapport qualité / prix
9,9,Khaled,WH804CP0T2ER8NAFAMZ,4,très bon produit


<h3>Étape 3: Analyse et sauvgarde du contenu HTML des pages de commentaires inconnues

<h4><li>Analyse du contenu HTML des pages de commentaires des inconnues

In [23]:
users_data_list_unknown =[]
for i in ids_list:
    URL ="https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+str(i)
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    table = soup.findAll('li', attrs = {'class':'-df -i-ctr -ptxs -m'})
    if(table):      
        for row in table: 
            user_note = row.find('p', attrs = {'class' : '-gy5 -mw-34 -r'}).text
            user_note_number = int(user_note[1:len(user_note)-1])
            for j in range(1, user_note_number+1):
                user ={}
                user["user_id"] = id_generator
                id_generator = id_generator+1
                user["user_name"] ="unknown"
                user["product_id"] = i
                user["user_note"] = row.text[0:1]
                user["user_comment"] = "No Comment"
                users_data_list_unknown.append(user)

<h4><li>Sauvgarde des données dans un fichier CSV

In [24]:

filename = 'users_ratings_unknown.csv'
with open(filename, 'w', newline='', encoding="utf-8") as f:
    w = csv.DictWriter(f,['user_id','user_name','product_id','user_note', 'user_comment'])
    w.writeheader()
    for user in users_data_list_unknown:
        w.writerow(user)

In [25]:
df = pd.read_csv("users_ratings_unknown.csv")
df.head(10)

,user_id,user_name,product_id,user_note,user_comment
0,6594,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
1,6595,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
2,6596,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
3,6597,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
4,6598,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
5,6599,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
6,6600,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
7,6601,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
8,6602,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment
9,6603,unknown,WH804CP0T2ER8NAFAMZ,5,No Comment


<h3>Étape 4: Analyse et sauvgarde du contenu HTML des pages de commentaires secondaires

<h4><li>Analyse du contenu HTML des pages de commentaires secondaires</li></h4>

In [ ]:
URL = "https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+"<product_id>/?page=<page_number>"
r = requests.get(URL)
#print(r.content)

In [ ]:
id_generator=76962
users_data_list_secondary =[]
for element in ids_list:
    for j in range(2,8):
        URL = "https://www.jumia.com.tn/catalog/productratingsreviews/sku/"+str(element)+"/?page="+str(j)
        r = requests.get(URL)
        #print(r.content)
        soup = BeautifulSoup(r.content, 'html5lib')
        table = soup.findAll('article',attrs = {'class':'-pvs -hr _bet'})
        if(table):
            for i in table:         
                user ={}
                user["user_id"] = id_generator
                id_generator = id_generator+1
                user_name_full =i.find('span', attrs = {'class':''}).text
                user["user_name"] =user_name_full[4:(len(user_name_full))]
                user["product_id"] = element
                user_note_full = i.find('div', attrs = {'class' : 'stars _m _al -mvs'}).text
                user["user_note"] = user_note_full[0:1]
                user["user_comment"] = i.find('h3').text
                users_data_list_secondary.append(user)
    print(ids_list.index(i))


<h4><li>Sauvgarde des données dans un fichier CSV

In [ ]:
filename = 'users_ratings_secondary.csv'
with open(filename, 'w', newline='', encoding="utf-8") as f:
    w = csv.DictWriter(f,['user_id','user_name','product_id','user_note', 'user_comment'])
    w.writeheader()
    for user in users_data_list_secondary:
        w.writerow(user)

In [5]:
df = pd.read_csv("users_ratings_secondary.csv")
df.head(10)

,user_id,user_name,product_id,user_note,user_comment
0,76962,Mourad,WH804CP0T2ER8NAFAMZ,5,Satisfait
1,76963,Heythem,WH804CP0T2ER8NAFAMZ,1,mauvaise qualité
2,76964,Besma,WH804CP0T2ER8NAFAMZ,5,J'aime
3,76965,SANDLI,WH804CP0T2ER8NAFAMZ,5,j'aime
4,76966,Ahmed,WH804CP0T2ER8NAFAMZ,4,J'aime ça
5,76967,Abdalbasset,WH804CP0T2ER8NAFAMZ,4,parfait.
6,76968,Omo,WH804CP0T2ER8NAFAMZ,2,moyen
7,76969,Nedra,WH804CP0T2ER8NAFAMZ,4,j'aime
8,76970,Binouza,WH804CP0T2ER8NAFAMZ,1,Je n'aime pas
9,76971,Mouna,WH804CP0T2ER8NAFAMZ,4,Très bon rapport qualité-prix


<h3>Étape 4: Collection des fichiers csv

In [13]:
file1 = open("users_ratings.csv", "a", encoding="utf-8")
file2 = open("users_ratings_unknown.csv", "r" ,encoding="utf-8")
file3 = open("users_ratings_secondary.csv", "r" ,encoding="utf-8")


for line in file2:   
    file1.write(line)
    
for line in file3:
    file1.write(line)

file1.close()
file2.close()
file3.close()


In [17]:
df = pd.read_csv("users_ratings.csv")
df.head(10)

,user_id,user_name,product_id,user_note,user_comment
0,0,Ayman,WH804CP0T2ER8NAFAMZ,5,J’aime ça
1,1,Fatima,WH804CP0T2ER8NAFAMZ,4,Produit chinois
2,2,Sedki,WH804CP0T2ER8NAFAMZ,4,J'aime
3,3,Salem,WH804CP0T2ER8NAFAMZ,5,J'aime
4,4,Mohamed Aymen,WH804CP0T2ER8NAFAMZ,4,bonne qualité
5,5,Iyed,WH804CP0T2ER8NAFAMZ,5,tres bienn
6,6,Alaeddine,WH804CP0T2ER8NAFAMZ,5,j'aime
7,7,Mohamed riadh,WH804CP0T2ER8NAFAMZ,5,Excellent
8,8,Abou Rouaa,WH804CP0T2ER8NAFAMZ,4,bon rapport qualité / prix
9,9,Khaled,WH804CP0T2ER8NAFAMZ,4,très bon produit
